In [19]:
import pandas as pd

In [20]:
#df = pd.read_csv('CustomerSegmentation.csv' , index_col = 0 , encoding = 'Unicode_escape' )

In [21]:
!pip install fastai

/bin/bash: /home/user/ENTER/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [22]:
 pip install psycopg2-binary

/bin/bash: /home/user/ENTER/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [26]:
from fastai.collab import *
from fastai.tabular.all import *

In [27]:
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine

In [28]:
dbname = "railway"
user = "postgres"
password = "*B1AcCDbDegA4DD5DfB*C-fG5a6-c1e6"
host = "roundhouse.proxy.rlwy.net"
port = "55650"

In [29]:
engine2 = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [30]:
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)


In [31]:

cur = conn.cursor()

In [32]:
table_name = "customer_sale"
query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(table_name))

In [33]:
# Execute the query
cur.execute(query)

In [34]:
# Fetch all rows
rows = cur.fetchall()

In [38]:
# Create a DataFrame from the fetched data
columns = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=columns)

In [39]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,20341.000000,20341.000000,20341.000000
mean,9.907576,3.122585,15582.221867
std,79.100245,5.287087,1730.719532
min,-9360.000000,0.000000,12347.000000
25%,1.000000,1.250000,14298.000000
50%,4.000000,2.100000,15596.000000
75%,12.000000,3.750000,17198.000000
max,2880.000000,295.000000,18269.000000


In [40]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20341 entries, 0 to 20340
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StockCode    20341 non-null  object 
 1   Description  20341 non-null  object 
 2   Quantity     20341 non-null  int64  
 3   InvoiceDate  20341 non-null  object 
 4   UnitPrice    20341 non-null  float64
 5   CustomerID   20341 non-null  float64
 6   Country      20341 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB


In [41]:
df = df.drop(['Description'], axis=1   )

In [42]:
df['Quantity'] = df['Quantity'].astype(int)

In [43]:
df = df.dropna()

In [44]:
quant = df.groupby(['CustomerID', 'StockCode'])['Quantity'].sum().reset_index()
quant = quant.rename(columns={'Quantity': 'totalquantity'})

In [45]:
quant.head()

,CustomerID,StockCode,totalquantity
0,12347.0,20780,12
1,12347.0,20782,6
2,12347.0,21064,6
3,12347.0,21171,12
4,12347.0,21731,12


In [46]:
import math

In [47]:
quant['totalquantity'] = quant['totalquantity'].astype(int)


In [48]:
quant['totalquantity'].max()

2880

In [49]:
for i in range(len(quant)) :
   quant.loc[i, 'totalquantity'] = abs(quant['totalquantity'].iloc[i])


In [50]:
dfm = quant 

In [51]:
dfm.head()

,CustomerID,StockCode,totalquantity
0,12347.0,20780,12
1,12347.0,20782,6
2,12347.0,21064,6
3,12347.0,21171,12
4,12347.0,21731,12


In [52]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18222 entries, 0 to 18221
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CustomerID     18222 non-null  float64
 1   StockCode      18222 non-null  object 
 2   totalquantity  18222 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 427.2+ KB


In [53]:
dls = CollabDataLoaders.from_df(dfm, item_name='StockCode', user_name='CustomerID', rating_name='totalquantity' , bs=64 , valid_pct= 0.2)
dls.show_batch()

,CustomerID,StockCode,totalquantity
0,14932.0,22112,3
1,13838.0,22316,1
2,16098.0,22726,4
3,16893.0,22147,1
4,12766.0,20685,2
5,13030.0,72802C,6
6,14723.0,22130,6
7,14800.0,22940,15
8,15024.0,84879,16
9,12647.0,21949,12


In [54]:
learner_inf = collab_learner(dls, y_range=(0, 100), layers=[100,50])
learner_inf.fit_one_cycle(8)

epoch,train_loss,valid_loss,time
0,4617.215820,4160.716309,00:01
1,7978.174316,3892.183838,00:00
2,6112.908203,3276.251953,00:00
3,2213.120117,2886.622314,00:00
4,16660.044922,2721.947021,00:00
5,2332.965820,2655.305176,00:00
6,12992.678711,2632.823486,00:00
7,27699.429688,2629.556641,00:00


In [55]:
learner_inf.model

EmbeddingDotBias(
  (u_weight): Embedding(751, 50)
  (i_weight): Embedding(2231, 50)
  (u_bias): Embedding(751, 1)
  (i_bias): Embedding(2231, 1)
)

In [56]:
user_bias = learner_inf.model.i_bias.weight
item_bias = learner_inf.model.u_bias.weight
#idxs = item_bias.argsort(descending=True)[:5]
#[dls.classes['StockCode'][i] for i in idxs]

In [57]:
#print(idxs)  #these are movie bias

In [58]:
sale = learner_inf.model.i_weight.weight.squeeze()

In [59]:
print(sale)

tensor([[ 0.0096, -0.0071, -0.0146,  ...,  0.0009,  0.0051,  0.0015],
        [-0.1586,  0.1348, -0.1432,  ..., -0.1249,  0.1375,  0.1349],
        [-0.0341,  0.0309, -0.0411,  ..., -0.0322,  0.0303,  0.0546],
        ...,
        [-0.0671,  0.0537, -0.0804,  ..., -0.0597,  0.0449,  0.0788],
        [-0.1670,  0.1004, -0.1518,  ..., -0.1417,  0.0722,  0.1173],
        [-0.2183,  0.1291, -0.2089,  ..., -0.2579,  0.2797,  0.2305]],
       grad_fn=<SqueezeBackward0>)


In [60]:
user = learner_inf.model.u_weight.weight.squeeze()

In [61]:
print(user)

tensor([[-3.1018e-03,  1.5689e-03, -7.7354e-03,  ...,  1.6474e-02,
         -8.2351e-03, -3.5403e-05],
        [ 2.3806e-01, -2.4420e-01,  2.4445e-01,  ...,  2.6476e-01,
         -1.8864e-01, -2.1960e-01],
        [ 2.8573e-01, -2.7602e-01,  2.7753e-01,  ...,  2.6183e-01,
         -1.3402e-01, -2.7698e-01],
        ...,
        [ 2.8489e-01, -2.3810e-01,  1.9790e-01,  ...,  1.7252e-01,
         -1.8863e-01, -1.6951e-01],
        [ 1.0361e-01, -5.1662e-02,  1.0561e-01,  ...,  1.2045e-01,
         -7.6411e-02, -8.5047e-02],
        [ 1.4031e-01, -1.2111e-01,  1.3110e-01,  ...,  1.3424e-01,
         -8.4368e-02, -1.2728e-01]], grad_fn=<SqueezeBackward0>)


In [62]:
sale_trans = sale.t()

In [63]:
mult = torch.matmul( user , sale_trans) 

In [64]:
mult.size()

torch.Size([751, 2231])

In [65]:
adding = add(mult , item_bias)

In [66]:
addingtra = adding.t()

In [67]:
adding2 = add(addingtra , user_bias)

In [68]:
final = adding2.t()

In [69]:
final.size()

torch.Size([751, 2231])

In [70]:
print(final)

tensor([[-0.0236, -0.1751, -0.0558,  ..., -0.0658, -0.1702, -0.2627],
        [-0.2677, -1.6587, -0.6182,  ..., -0.8181, -1.6360, -2.3708],
        [-0.3338, -1.9667, -0.7615,  ..., -0.9970, -1.9539, -2.8734],
        ...,
        [-0.2803, -1.7957, -0.6460,  ..., -0.8731, -1.7566, -2.5558],
        [-0.0929, -0.8317, -0.2743,  ..., -0.3909, -0.8160, -1.2451],
        [-0.1613, -1.0672, -0.3816,  ..., -0.5210, -1.0505, -1.5470]],
       grad_fn=<TBackward0>)


In [71]:
out, inds = torch.max(final,dim=1) #since dim =1 ; we will get max of each row

In [72]:
print(inds)

tensor([1923, 1923,   28, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1350,
          28, 1923, 1923, 1923, 1923,  234, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923, 1923,   28,   28, 1923, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923, 1923,   28,  984, 1438, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923,   28, 1923,   28, 1923, 1923, 1923, 1923, 1923,   64, 1923,
        1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923, 1923, 1923, 1923, 1923,   28, 1923, 1923, 1923, 1923, 1923,
        1350, 1802, 1923, 1350, 1923,   28, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923, 1923,   28, 1923, 1923, 1923, 1923, 1802, 1923, 1923, 1923,
        1923, 1923, 1689, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923,
        1251, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923,
        1923, 1923, 1923, 1923, 1923, 1923,   28, 1923, 1923, 1923, 1438, 1923,
        1923, 1438, 1438, 1923, 1923, 19

In [73]:
dfd = pd.DataFrame()

In [74]:
dfd['bestsale'] = 'yh'

In [75]:
dfd['customerid'] = '0'

In [76]:
dfd.head()

,bestsale,customerid


In [79]:
for i in range(len(inds)) :
  dfd['bestsale'].loc[i] = dfm['StockCode'].iloc[int(inds[i])]

In [81]:
dfd1 = dfm['CustomerID'].unique()

In [82]:
print(dfd1)

[12347. 12370. 12386. 12395. 12427. 12429. 12431. 12433. 12434. 12441.
 12471. 12472. 12474. 12476. 12481. 12494. 12540. 12557. 12567. 12583.
 12586. 12600. 12605. 12621. 12647. 12649. 12662. 12681. 12682. 12683.
 12686. 12708. 12709. 12712. 12720. 12721. 12725. 12731. 12738. 12747.
 12748. 12753. 12755. 12763. 12766. 12779. 12785. 12791. 12793. 12797.
 12808. 12826. 12838. 12839. 12841. 12855. 12868. 12870. 12872. 12875.
 12913. 12915. 12921. 12942. 12947. 12963. 12967. 12971. 12989. 13000.
 13008. 13011. 13013. 13021. 13030. 13034. 13047. 13050. 13065. 13069.
 13077. 13078. 13081. 13089. 13090. 13093. 13094. 13102. 13108. 13113.
 13115. 13117. 13126. 13136. 13138. 13140. 13145. 13148. 13155. 13173.
 13174. 13178. 13198. 13199. 13209. 13211. 13225. 13230. 13240. 13255.
 13267. 13269. 13270. 13295. 13305. 13319. 13324. 13327. 13329. 13369.
 13370. 13402. 13408. 13418. 13448. 13455. 13468. 13481. 13488. 13491.
 13495. 13497. 13506. 13520. 13523. 13531. 13564. 13569. 13576. 13579.
 13599

In [83]:
for i in range(len(dfd1)) :
    dfd['customerid'].loc[i] = dfd1[i]

In [84]:
dff = pd.DataFrame(columns=['customerid'])

In [85]:
dff = dff.rename(columns={'customerid': 'customerid'})

In [86]:
for row in dfd1:
    dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)

/tmp/ipykernel_5887/878400216.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
/tmp/ipykernel_5887/878400216.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
/tmp/ipykernel_5887/878400216.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
/tmp/ipykernel_5887/878400216.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dff = dff.append(pd.Series(row, index=dff.columns), ignore_index=True)
/tmp/ipykernel_5887/878400216.py

In [87]:
dff.head()

,customerid
0,12347.0
1,12370.0
2,12386.0
3,12395.0
4,12427.0


In [88]:
dff.describe()

,customerid
count,761.000000
mean,15517.507227
std,1714.882721
min,12347.000000
25%,14085.000000
50%,15605.000000
75%,17026.000000
max,18269.000000


In [89]:
dff1 = pd.DataFrame(columns=['sale'])

In [90]:
arr = np.empty(len(inds), dtype=object)

In [92]:
for i in range(len(inds)):
     arr[i] = dfm['StockCode'].iloc[int(inds[i])]

In [93]:
#print(arr)

In [94]:
#print(len(arr))

In [95]:
dff1 = pd.DataFrame({'stockcode': arr})


In [96]:
dff1.head()

,stockcode
0,85183B
1,85183B
2,85116
3,85183B
4,85183B


In [98]:
dff1.describe()

,stockcode
count,751
unique,19
top,85183B
freq,604


In [99]:
result = pd.concat([dff, dff1], axis=1 )

In [100]:
result.head()

,customerid,stockcode
0,12347.0,85183B
1,12370.0,85183B
2,12386.0,85116
3,12395.0,85183B
4,12427.0,85183B


In [101]:
# Get the indices of the second-largest elements along each row
index2, indices_second_largest = torch.topk(final, k=2, dim=1)

# Use the indices to get the second-largest elements
second_largest_elements = torch.gather(final, dim=1, index=indices_second_largest[:, 1].view(-1, 1))

In [102]:

# Convert the PyTorch tensor to a NumPy array
indices_second_largest_np = indices_second_largest.detach().numpy()
second_largest_indices = indices_second_largest_np[:, 1]


In [103]:
print(second_largest_indices)

[1811 1229 1350 1229   28   28 1350   28 1887 1350   28   28 1923 1350
   28   28   28 1802   28   28 1350 1350   28   28 1350   28   28 1923
 1923 1350   28   28   28   28   28 1229   28   28 1350   59 1438  984
 1350   28 1350   28 1350   28   59   28 1923   28 1923   28 1887   28
 1350   28 1923 1350   28   28   28   28   28 1229   28 1350   28 1887
 1350   28 1811   28 1350 1350   28   28 1923   28 1350   28   28 1350
 1923 1438   28 1923   28 1923   28 1350   28   59 1811   28   28   28
 1798 1923   28 1968   28 1350  234   28   28 1350   28   28 1162 1350
   59   28 1483   28 1350   28   28   28 1802   28   28   28   28 1350
   28   28   28   28   28   28   28   28   28   28   28   28 1923   28
   28   28 2064   28   28 1802 1802 1229 1887   28  234   28   28  329
 1350   28 1350   28   28 1438 1350   28  984 1887 1350 1229   28   28
   28   28 1350   28   28   28 1350   28   28 1802   28   28   28 1162
 1350 1350   28 1923 1229 1923   28 1229 1923 1887 1923   28   28   28
   28 

In [104]:

# Convert the PyTorch tensor to a NumPy array
second_largest_elements_np = second_largest_elements.detach().numpy()

# Create a new DataFrame or add a new column to the existing DataFrame
#result['2best'] = second_largest_elements_np.tolist()

In [105]:
result['bestsale2nd'] = result['customerid'] + 5

In [110]:
for i in range(len(inds)):
    result.loc[i, 'bestsale2nd'] = dfm['StockCode'].iloc[int(second_largest_indices[i])]

In [111]:
result.head()

,customerid,stockcode,bestsale2nd
0,12347.0,85183B,20750
1,12370.0,85183B,72709
2,12386.0,85116,21080
3,12395.0,85183B,72709
4,12427.0,85183B,85116


In [112]:
##THIRD LARGEST INDEXX

In [113]:
# Get the indices of the second-largest elements along each row
index3, indices_third_largest = torch.topk(final, k=3, dim=1)

In [114]:

# Convert the PyTorch tensor to a NumPy array
indices_third_largest_np = indices_third_largest.detach().numpy()
third_largest_indices = indices_third_largest_np[:, 2]

In [115]:
print(third_largest_indices)

[1350   28 1923 1483 1350   59   28   59 1483 1887 1350 1923 1350  304
 1350 1350 1350 1103 1798 1350   28   28 1350   59 1483 1229 1350   59
 1229   28   59 1229 1350 1229 1350 1350   59 1350   28 1350 1162 1802
   28 1798   28 1229 1887 1229   28   59   59 1798 1350 1350 1483 1229
 1483   59 1350 1483   59 1350   59 1350   59 1887   59 1483 1229 1483
   28 1229 1350 1229   28   28   59   59   59 1483   28   59 1887   28
   28 2076 1350   28 1350   59 1350   28   59   28 1887   59 1798 1350
 1229   59   59 1811 1350   28  825   59   59 1483 1350 1229  984 1887
 1887   59 1887   59   28 1350 1229   59 1103 1229 1798   59 1350 1483
 1350   59 1229 1350 1350 1350 1229 1229   59   59 1350 1229 1350   59
 1229 1350 1162 1350 1798 1162 1102 1483 1483 1350  460 1229 1350 1923
   28   59   28 1798   59 1235   28 1350 2074 1229 1483   28 1350   59
 1229 1229 1483 1350   59   59   28 1350   59 2064 1350 1350   59 1811
   28   59 1798 1350   28 1798 1350   28 1229 1483 1798   59 1350   59
 1887 

In [116]:
result['bestsale3rd'] = result['customerid'] + 5

In [117]:
for i in range(len(inds)):
    result.loc[i, 'bestsale3rd'] = dfm['StockCode'].iloc[int(third_largest_indices[i])]

In [118]:
result.head()

,customerid,stockcode,bestsale2nd,bestsale3rd
0,12347.0,85183B,20750,21080
1,12370.0,85183B,72709,85116
2,12386.0,85116,21080,85183B
3,12395.0,85183B,72709,21976
4,12427.0,85183B,85116,21080


In [120]:
result.head()

,customerid,stockcode,bestsale2nd,bestsale3rd
0,12347.0,85183B,20750,21080
1,12370.0,85183B,72709,85116
2,12386.0,85116,21080,85183B
3,12395.0,85183B,72709,21976
4,12427.0,85183B,85116,21080


In [121]:
result = result.dropna()

In [122]:
table_name = 'customerrecostock'

In [123]:
with engine2.connect() as connection:
    connection.execute(f"DELETE FROM {table_name}")

In [124]:
result.to_sql(table_name, engine2, index=False, if_exists='append')

751

In [ ]:
#result.to_csv('finaloutput_recommender2.csv')